# Cerebro

#### <font color='blue'>Imports</font>

In [ ]:
from cerebro.etl.etl_spec import ETLSpec
from cerebro.experiment import Experiment
from cerebro.mop.sub_epoch_spec import SubEpochSpec

### <font color='blue'> Initialize Data Preprocessing </font>

In [ ]:
class ImagenetETLSpec(ETLSpec):
    def __init__(self):
        self.is_feature_download = [False, False, True, False, False]
        
    def initialize_worker(self):
        pass
    
    def row_preprocessor(self, row, mode, object_dir):
        import torch
        import numpy as np
        from PIL import Image

        input_image_path = object_dir + "/" + str(row["filepath"])

        pil_image = Image.open(input_image_path)
        image = np.asarray(pil_image.convert('RGB').resize((112, 112)))
        image = image / 255.0
        image = image - [0.485, 0.456, 0.406]
        image = image / [0.229, 0.224, 0.225]

        torch_image = torch.from_numpy(image).float()
        image = torch.reshape(torch_image, (torch_image.shape[2], torch_image.shape[0], torch_image.shape[1]))
        if mode == 'test':
            return image, None
        else:
            label = torch.tensor(row["label"])
            return image, label

### <font color='blue'> Initialize Model Building </font>

In [ ]:
class ImagenetTrainingSpec(SubEpochSpec):
    def __init__(self):
        import torch
        import torch.nn as nn

        self.criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()
        self.log_softmax = torch.nn.LogSoftmax().cuda() if torch.cuda.is_available() else torch.nn.LogSoftmax()

    def initialize_worker(self):
        pass

    def create_model_components(self, hyperparams):
        import torch
        from torchvision import models

        learning_rate = hyperparams["learning_rate"]
        lambda_value = hyperparams["lambda_value"]
        model_type = hyperparams["model_type"]

        if model_type == "resnet50":
            model = models.resnet50(pretrained=False)
        elif model_type == "vgg16":
            model = models.vgg16(pretrained=False)

        # Define the optimizer
        optimizer = torch.optim.Adam(model.parameters(),
                                         lr=learning_rate,
                                         weight_decay=lambda_value)
        
        model_object = {
            "imagenet_model": model,
            "optimizer": optimizer
        }

        return model_object

    def accuracy(self, output, target, topk=(1, ), binary=False):
        import torch

        """Computes the precision@k for the specified values of k"""
        if binary:
            batch_size = target.size(0)
            _, pred = torch.max(output.data, 1)
            correct = (pred == target).sum().item()
            res = [torch.tensor(correct / batch_size)]
        else:
            maxk = max(topk)
            maxk = min(maxk, output.shape[1])
            batch_size = target.size(0)

            _, pred = output.topk(maxk, 1, True, True)
            pred = pred.t()
            correct = pred.eq(target.view(1, -1).expand_as(pred))
            res = []
            for k in topk:
                correct_k = correct[:k].reshape(-1).float().sum(0)
                res.append(correct_k.mul_(1.0 / batch_size))
        return res

    def train(self, model_object, dataloader, hyperparams, input_device, output_device):
        import math
        import torch

        train_metrics = []
        batch_size = hyperparams["batch_size"]

        obj = model_object("load")
        model = obj["imagenet_model"]
        optimizer = obj["optimizer"]
        
        model.train()

        i_step = 0
        subepoch_total_step = math.ceil(len(dataloader.dataset) / batch_size)

        for batch in dataloader:
            images, labels = batch[0].to(input_device), torch.tensor(batch[1]).to(output_device)
            outputs = model(images)
            loss = self.criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            model.zero_grad()
            optimizer.zero_grad()

            outputs_softmax = self.log_softmax(outputs)
            
            top_1_acc, top_5_acc = self.accuracy(outputs_softmax, labels, (1, 5))
            
            subepoch_loss = loss.item() / batch_size
            subepoch_top_1_acc = top_1_acc.item()
            subepoch_top_5_acc = top_5_acc.item()

            stats_dict = {
                "subepoch_loss": subepoch_loss,
                "subepoch_top_1_acc": subepoch_top_1_acc,
                "subepoch_top_5_acc": subepoch_top_5_acc
            }

            stats = "Train step [%d/%d], subepoch_loss: %.4f, subepoch_top_1_acc: %5.4f, , subepoch_top_5_acc: %5.4f" \
                        % (i_step, subepoch_total_step,subepoch_loss, 
                           subepoch_top_1_acc, subepoch_top_5_acc)

            print("\r" + stats, end="")

            train_metrics.append(stats_dict)

            i_step += 1

        model_object("save", {
                    "imagenet_model": model,
                    "optimizer" : optimizer
                   })
        return train_metrics
    
    def val(self, model_object, dataloader, hyperparams, input_device, output_device):
        import math
        import torch

        batch_size = hyperparams["batch_size"]

        obj = model_object("load")
        model = obj["imagenet_model"]
        
        model.eval()

        epoch_loss = 0.0
        epoch_top_1_acc = 0.0
        epoch_top_5_acc = 0.0
        subepoch_total_step = math.ceil(len(dataloader.dataset) / batch_size)
        
        batch_num = 1
        with torch.no_grad():    
            for batch in dataloader:
                images, labels = batch[0].to(input_device), torch.tensor(batch[1]).to(output_device)
                outputs = model(images)
                loss = self.criterion(outputs, labels)
                outputs_softmax = self.log_softmax(outputs)
                top_1_acc, top_5_acc = self.accuracy(outputs_softmax, labels, (1, 5))

                epoch_loss += (loss.item() / batch_num )
                epoch_top_1_acc += (top_1_acc.item() / batch_num)
                epoch_top_5_acc += (top_5_acc.item() / batch_num)
            
                batch_num += 1

                stats = "Validation step [%d/%d], loss: %.4f, top_1_acc: %5.4f, , top_5_acc: %5.4f" \
                            % (batch_num, subepoch_total_step,epoch_loss, 
                            epoch_top_1_acc, epoch_top_5_acc)

            print("\r" + stats, end="")

        val_metrics = {
            "total_epoch_loss": epoch_loss,
            "total_epoch_top_1_acc": epoch_top_1_acc,
            "total_epoch_top_5_acc": epoch_top_5_acc
        }
    
        return val_metrics
    
    def predict(self, model_object, dataloader, hyperparams, input_device):
        import math
        import torch

        predictions = []
        batch_size = hyperparams["batch_size"]

        obj = model_object("load")
        model = obj["imagenet_model"]
        
        model.eval()

        subepoch_total_step = math.ceil(len(dataloader.dataset) / batch_size)
        
        batch_num = 1
        with torch.no_grad():    
            for batch in dataloader:
                images = batch[0].to(input_device)
                outputs = model(images)
                outputs_softmax = self.log_softmax(outputs)
                predictions.append(outputs_softmax)

                batch_num += 1

                stats = "Test step [%d/%d]" \
                            % (batch_num, subepoch_total_step)

            print("\r" + stats, end="")
    
        return predictions

#### <font color='blue'> Model Building specifications </font>

In [ ]:
num_epochs = 2
param_grid = {
    'batch_size': [128, 256],
    'learning_rate': [1e-2, 1e-3],
    'lambda_value': [1e-3, 1e-4],
    'model_type': ['vgg16', 'resnet50']
}

#### <font color='blue'> Initialize Experiment </font>

In [ ]:
experiment = Experiment()
imagenet_etl_spec = ImagenetETLSpec()
imagenet_training_spec = ImagenetTrainingSpec()

#### <font color='blue'> Run Data Preprocessing </font>

In [ ]:
experiment.run_etl(imagenet_etl_spec, fraction=0.1)

#### <font color='blue'> Run Model Building </font>

In [ ]:
experiment.run_fit(imagenet_training_spec, param_grid, num_epochs)

In [ ]:
experiment.run_predict()